In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_CLASSES = 22


In [2]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    'skin_diseases/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    'skin_diseases/train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'skin_diseases/test',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 13898 images belonging to 22 classes.
Found 13898 images belonging to 22 classes.
Found 1546 images belonging to 22 classes.


In [3]:
base_model = EfficientNetB0(include_top=False, weights='imagenet', input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))
base_model.trainable = False  # Freeze initial layers

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


In [4]:
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=val_generator
)


c:\Users\manasvi\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 697s 2s/step - accuracy: 0.2280 - loss: 2.6514 - val_accuracy: 0.4380 - val_loss: 1.8884
Epoch 2/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 616s 1s/step - accuracy: 0.3767 - loss: 2.0471 - val_accuracy: 0.4791 - val_loss: 1.7249
Epoch 3/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 532s 1s/step - accuracy: 0.4007 - loss: 1.9429 - val_accuracy: 0.4930 - val_loss: 1.6617
Epoch 4/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 535s 1s/step - accuracy: 0.4245 - loss: 1.8731 - val_accuracy: 0.5001 - val_loss: 1.6230
Epoch 5/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 516s 1s/step - accuracy: 0.4443 - loss: 1.7930 - val_accuracy: 0.5242 - val_loss: 1.5472
Epoch 6/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 510s 1s/step - accuracy: 0.4373 - loss: 1.7916 - val_accuracy: 0.5348 - val_loss: 1.5095
Epoch 7/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 514s 1s/step - accuracy: 0.4607 - loss: 1.7423 - val_accuracy: 0.5502 - val_loss: 1.4643
Epoch 8/15
435/435 ━━━━━━━━━━━━━━━━━━━━ 508s 1s/step - accuracy: 0.4650 - loss: 1.7390 - val_accu

In [5]:
base_model.trainable = True

# Optionally freeze first few layers:
for layer in base_model.layers[:150]:
    layer.trainable = False

model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

history=model.fit(train_generator, validation_data=val_generator, epochs=25)


Epoch 1/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 683s 2s/step - accuracy: 0.3372 - loss: 3.0064 - val_accuracy: 0.5510 - val_loss: 1.4200
Epoch 2/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 664s 2s/step - accuracy: 0.4746 - loss: 1.7023 - val_accuracy: 0.6032 - val_loss: 1.2620
Epoch 3/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 645s 1s/step - accuracy: 0.5196 - loss: 1.5329 - val_accuracy: 0.6281 - val_loss: 1.1785
Epoch 4/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 642s 1s/step - accuracy: 0.5491 - loss: 1.4233 - val_accuracy: 0.6602 - val_loss: 1.0698
Epoch 5/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 1197s 3s/step - accuracy: 0.5769 - loss: 1.3488 - val_accuracy: 0.6854 - val_loss: 0.9887
Epoch 6/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 2531s 6s/step - accuracy: 0.5942 - loss: 1.2832 - val_accuracy: 0.7146 - val_loss: 0.9182
Epoch 7/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 623s 1s/step - accuracy: 0.6169 - loss: 1.2014 - val_accuracy: 0.7279 - val_loss: 0.8627
Epoch 8/25
435/435 ━━━━━━━━━━━━━━━━━━━━ 1076s 2s/step - accuracy: 0.6315 - loss: 1.1462 - val_a

In [10]:
model.save("skin_disease_model.h5")
model.save("skin_disease_model.keras")